# Phase 2i: Running IQtree

Generates distance tree using IQTree.

```
Parameters
-------------
save_dir: str
    Path to directory for saving outputs in.

cache_dir: str 
    Path to directory for cached objects in.

fasta_file: str
    Name used for fasta files containing sequences.

partition: str
    Name of partition to use when calling `sbatch`.
```

In [ ]:
save_dir='runs_of_pipeline/2025-02-05'
cache_dir='cache'
fasta_file= 'sequences_with_root.fasta'
partition=None

In [ ]:
# For some reason this still has to be called so that slurm's sbatch to use the beast_pype environment.
source activate beast_pype

In [ ]:
 job_ids_file=$cache_dir/slurm_job_ids.txt
 if [ -f "${fasta_file}" ]; then
    job_name="iqtree"
    job_id=$(sbatch -p $partition -J  $job_name -o "${save_dir}/iqtree.out" -c 11 --mem=32G --parsable --time=1-00:00:00\
        --wrap="iqtree -ninit 2 -n 2 -me 0.05 -nt 11 -s ${fasta_file} -m GTR -ninit 10 -n 4 -pre ${save_dir}/iqtree; echo 'slurm_job_complete'")
    echo "${job_id}" >> $job_ids_file
else
     for i in $save_dir/*
        do
            if [ -d $i ] && [ "$i" != "$cache_dir" ]; then
                xml_set=${i##*/}
                job_name="${xml_set}_iqtree"
                job_id=$(sbatch -p $partition -J  $job_name -o "${i}/iqtree.out" -c 11 --mem=32G --parsable --time=1-00:00:00\
                    --wrap="iqtree -ninit 2 -n 2 -me 0.05 -nt 11 -s ${i}/${fasta_file} -m GTR -ninit 10 -n 4 -pre ${i}/iqtree; echo 'slurm_job_complete'")
                echo "${job_id}" >> $job_ids_file
            fi
        done
fi

To check the progress of the slurm jobs building IQTrees use the terminal command: `squeue --me`.